# Week 1 - January 23

## **Monday 01/02**

### WSL and X-session

I have left on-going tests last week of 22.

Trying to start a X-session on ubuntu 22.04 within this new version of WSL.


**systemctl-sysusers failed**

seen in `journalctl -r` when starting `gnome-session` some errors linked to `systemd-sysusers.service`.

And I can see failed services with

```bash
sudo systemctl --failed
  UNIT                     LOAD   ACTIVE SUB    DESCRIPTION
● systemd-sysusers.service loaded failed failed Create System Users

LOAD   = Reflects whether the unit definition was properly loaded.
ACTIVE = The high-level unit activation state, i.e. generalization of SUB.
SUB    = The low-level unit activation state, values depend on unit type.
1 loaded units listed.
```